In [1]:
import pandas as pd
import string

ttl_data = pd.read_json('tracktranslegislation.json')
aclu_data = pd.read_json('aclu.json')
aclu_data['state'] = aclu_data['state'].transform(lambda x: x['value'])

In [3]:
aclu_bills = aclu_data.copy()
#aclu_bills['state'] = aclu_bills['state'].transform(lambda x: x['value'])
aclu_bills['name'] = aclu_bills['name'].transform(lambda x: x.replace(' ', '').replace('23-', ''))
aclu_bills = aclu_bills[['state', 'name']]

ttl_bills = ttl_data[['state', 'billId']].rename(columns={'billId': 'name'})
ttl_bills

merged = aclu_bills.merge(ttl_bills, indicator=True, how='outer')
# merged[merged['_merge'] == 'left_only']
pd.set_option('display.max_rows', None)
merged[merged['_merge'] != 'both'].sort_values(by=['state', 'name'])

aclu_only = merged[merged['_merge'] == 'left_only']
def match_suffix(num: str, billId: str):
    if not billId.endswith(num):
        return False
    
    search = billId.rstrip(num)[::-1]
    for item in search:
        if item in '123456789':
            return False
    
    return True

def advanced_check(row):
    state = row['state']
    ttl_for_state = list(ttl_bills.loc[ttl_bills.state == state]['name'])
    prefix, num = row['name'].split(' ')
    prefix = prefix.strip(string.punctuation)
    candidates = sorted([
        bill 
        for bill 
        in ttl_for_state 
        if (
            bill[0] == prefix[0]
            and match_suffix(num, bill)
        )
    ])
    
    return len(candidates) == 0

#for idx, row in aclu_data.loc[aclu_only.index].iterrows():
#    if advanced_check(row):
#        pass
#        print('\t'.join([state, prefix, num, '|',  *candidates]))
        

# match_suffix('1674', 'S1674')
first_filter = aclu_data.loc[aclu_only.index]
second_filter = first_filter[first_filter.apply(lambda x: advanced_check(x), axis=1)]
second_filter.sort_values(['state', 'name'])

print(len(aclu_data))
print(len(ttl_data))
print(len(second_filter))
second_filter

408
372
114


,name,state,link,companion_bill,impact,status,status_description,status_date
2,HB 265,ID,https://legislature.idaho.gov/sessioninfo/2023...,,"[{'value': 'speech', 'label': 'Free Speech & E...","{'value': 'advancing', 'label': 'Advancing'}",Passed House; Senate read referred to committee,03/08/2023
3,SB 17,TX,https://capitol.texas.gov/BillLookup/History.a...,,"[{'value': 'other', 'label': 'Other Anti-LGBTQ...","{'value': 'advancing', 'label': 'Advancing'}",Referred to committee,03/13/2023
4,SB 288,NV,https://www.leg.state.nv.us/App/NELIS/REL/82nd...,,"[{'value': 'other', 'label': 'Other Anti-LGBTQ...","{'value': 'advancing', 'label': 'Advancing'}",Referred to committee,03/09/2023
5,SB 2199,TX,https://capitol.texas.gov/BillLookup/History.a...,,"[{'value': 'rights', 'label': 'Civil Rights'}]","{'value': 'introduced', 'label': 'Introduced'}",Filed,03/09/2023
8,SB 8,TX,https://capitol.texas.gov/BillLookup/History.a...,,"[{'value': 'school', 'label': 'Schools & Educa...","{'value': 'advancing', 'label': 'Advancing'}",Referred to committee,03/13/2023
14,SB 96,AK,https://www.akleg.gov/basis/Bill/Detail/33?Roo...,HB 105,"[{'value': 'school', 'label': 'Schools & Educa...","{'value': 'advancing', 'label': 'Advancing'}",First read and referred to committee,03/08/2023
15,HB 105,AK,https://www.akleg.gov/basis/Bill/Detail/33?Roo...,SB 96,"[{'value': 'school', 'label': 'Schools & Educa...","{'value': 'advancing', 'label': 'Advancing'}",First read and referred to committee,03/08/2023
16,HB 4624,TX,https://capitol.texas.gov/BillLookup/History.a...,,"[{'value': 'health', 'label': 'Healthcare'}]","{'value': 'introduced', 'label': 'Introduced'}",Filed,03/09/2023
19,HB 138,PA,https://www.legis.state.pa.us/cfdocs/billinfo/...,,"[{'value': 'health', 'label': 'Healthcare'}]","{'value': 'advancing', 'label': 'Advancing'}",Referred to committee,03/08/2023
25,SF 538,IA,https://www.legis.iowa.gov/legislation/BillBoo...,,"[{'value': 'other', 'label': 'Other Anti-LGBTQ...","{'value': 'advancing', 'label': 'Advancing'}",Passed committee,03/06/2023
